In [23]:
from rockit import *
import numpy as np
import matplotlib.pyplot as plt

Problem setup:

In [24]:
ocp = Ocp(T=(FreeTime(0.0)))

x1 = ocp.state()
x2 = ocp.state()
x3 = ocp.state()

u = ocp.control()

Parameters:

In [25]:
mass_rider = 78
mass_bike = 8
m = mass_rider + mass_bike
g = 9.81
my = 0.004
b0 = 0.091
b1 = 0.0087
Iw = 0.14
r = 0.33
Cd = 0.7
rho = 1.2
A = 0.4
eta = 1
w_prime = 26630
cp = 265
track_length = 1000
s = 0.05


Derivatives:

In [26]:
def sigmoid(x, x0, a):
    return 1/(1 + np.power(np.e, (-(x-x0)/a)))

In [27]:
ocp.set_der(x1, x2)
ocp.set_der(x2, 1/((x2)*(m + Iw/r**2)) * (eta*u - m*g*x2*s - my*m*g*x2 - b0*x2 - b1*x2**2 - 0.5*Cd*rho*A*x2**3))
ocp.set_der(x3, -(u-cp))
#ocp.set_der(x3, -(u-cp)*(1-sigmoid(u, cp, 3)) + (1-x3/w_prime)*(cp-u)*sigmoid(u, cp, 3))

Set objectives:

In [28]:
ocp.add_objective(ocp.T)

Add boundary constraints:

In [29]:
ocp.subject_to(ocp.at_t0(x1) == 0)
ocp.subject_to(ocp.at_t0(x2) == 1)
ocp.subject_to(ocp.at_t0(x3) == w_prime)
ocp.subject_to(ocp.at_tf(x1) == track_length)
#ocp.subject_to(ocp.at_t0(u) == 0)

Add path constraints:

In [30]:
ocp.subject_to(0 <= (u <= 500))
ocp.subject_to(0.1 <= (x2 <= 20))
ocp.subject_to(0 <= (x3 <= w_prime))
#ocp.subject_to(0 <= x3)
ocp.subject_to(0 <= (ocp.T <= 400))

Initial guesses:

In [31]:
ocp.set_initial(x2, 10)
ocp.set_initial(u, 250)
ocp.set_initial(ocp.T, 200)

Setup the solver:

In [32]:
opts = {"expand": True,
        "verbose": False,
        "print_time": True,
        "error_on_fail": True,
        "ipopt": {"linear_solver": "mumps",  # "ma57" is faster!
                  "max_iter": 5000,
                  'print_level': 5,
                  'sb': 'yes',  # Suppress IPOPT banner
                  'tol': 1e-5,
                  'hessian_approximation': 'limited-memory'
                  }}
ocp.solver("ipopt", opts)

method = MultipleShooting(N=10, intg='rk')
ocp.method(method)

In [33]:
sol = ocp.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      134
Number of nonzeros in inequality constraint Jacobian.:       35
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       44
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       34
Total number of inequality constraints...............:       35
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:       34
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0000000e+02 2.66e+04 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

 103  1.7103970e+02 9.49e+02 1.23e+03   1.4 8.68e+04    -  1.06e-03 1.59e-06f  2
 104  1.6940659e+02 9.47e+02 1.25e+03   1.4 5.26e+04    -  2.56e-02 1.66e-03h  1
 105  1.6887031e+02 9.46e+02 1.27e+03   1.4 1.10e+05    -  1.53e-03 3.50e-04h  1
 106  1.6884826e+02 9.46e+02 2.29e+04   1.4 4.43e+05    -  2.53e-03 1.20e-05h  1
 107r 1.6884826e+02 9.46e+02 9.99e+02   3.0 0.00e+00    -  0.00e+00 4.66e-07R 10
 108r 1.7058162e+02 7.22e+02 1.16e+03   2.8 1.81e+05    -  1.63e-02 2.26e-03f  1
 109  1.7057650e+02 7.22e+02 1.01e+04   3.1 1.35e+05    -  1.06e-02 2.76e-06f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  1.7040415e+02 7.23e+02 3.06e+04   3.9 1.48e+06    -  1.45e-03 2.60e-04f  2
 111  1.6988321e+02 7.28e+02 1.52e+06   3.6 1.51e+06    -  4.01e-03 9.11e-04h  1
 112  1.6978248e+02 7.28e+02 1.79e+06   5.0 2.17e+06    -  1.07e-02 9.48e-05h  3
 113  1.6940760e+02 7.28e+02 4.50e+06  -2.1 2.04e+06    -  1.36e-03 3.47e-04h  1
 114  1.6937322e+02 7.28e+02

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:157:
Error in Function::call for 'solver' [IpoptInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'solver' [IpoptInterface] at .../casadi/core/function.cpp:330:
.../casadi/core/nlpsol.cpp:862: nlpsol process failed. Set 'error_on_fail' option to false to ignore this error.

In [ ]:
tsol, usol = sol.sample(u, grid='integrator', refine=100)
plt.plot(tsol, usol)
plt.ylim(0,550)
plt.show()

In [ ]:
print(usol)

In [ ]:
tsol, x2sol = sol.sample(x2, grid='integrator', refine=1000)
plt.plot(tsol, x2sol)
plt.show()

In [ ]:
tsol, x3sol = sol.sample(x3, grid='integrator', refine=100)
plt.plot(tsol, x3sol)
plt.show()

In [ ]:
tf = sol.sample(ocp.T, grid="control")
print("Final time: ", tf[-1][-1])

In [ ]:
ocp.spy()